In [1]:
##Using Base environment

import numpy as np
import pickle as pkl
from pprint import pprint
import os
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
plt.ion()


import graspologic.utils as graspologic_utils
from graspologic.embed import AdjacencySpectralEmbed

from graspologic.datasets import load_drosophila_right
from graspologic.plot import heatmap
from graspologic.utils import binarize, symmetrize
import graspologic.utils as graspologic_utils
from scipy.linalg import orthogonal_procrustes
from graspologic.plot import heatmap
from sklearn.manifold import Isomap
#import piecewise_regression
from sklearn.linear_model import LinearRegression


#from kneed import DataGenerator, KneeLocator

def filter_matrix_TC(data, well):

    # Extract necessary data from the loaded data structure
    adj_matrix = data[well]['win_0']['adj_matrix_predicted']
    votes = data[well]['win_0']['votes']  # This variable is loaded but not used in the snippet you provided
    corr_peaks = data[well]['win_0']['corr_peaks']
    fs = data['config']['data']['fs']  # Sampling frequency

    # Initialize a matrix to track synchronization based on correlation peaks
    synced_matrix = np.full(adj_matrix.shape, False)
    for key in corr_peaks.keys():
        if np.all(np.abs(np.array(corr_peaks[key]['delays'])) < 1/fs):
            synced_matrix[key[0], key[1]] = True
            synced_matrix[key[1], key[0]] = True

    # Create the filtered matrix as per the given logic
    filtered_matrix = np.logical_and(adj_matrix, np.logical_not(synced_matrix))

    return filtered_matrix


import warnings

def find_optimal_neighbors(X):
    n_neighbors = 2
    while True:
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("error")
                isomap = Isomap(n_neighbors=n_neighbors)
                isomap.fit(X)
                print(f"Successful with n_neighbors={n_neighbors}")
                return n_neighbors
        except Warning as w:
            print(f"Warning encountered with n_neighbors={n_neighbors}: {w}")
            n_neighbors += 1




from scipy.stats import norm

def get_elbows(dat, n=3, threshold=False, plot=True, main=""):
    """
    Given a decreasingly sorted vector, return the given number of elbows.

    Args:
        dat: an input vector (e.g. a vector of standard deviations) or an input feature matrix.
        n: the number of returned elbows.
        threshold: either False or a number. If threshold is a number, then all
                   the elements in dat that are not larger than the threshold will be ignored.
        plot: logical. When True, it depicts a scree plot with highlighted elbows.
        main: title for the plot.

    Returns:
        q: a list of length n containing the positions of the elbows.
    """
    
    if isinstance(dat, np.ndarray) and len(dat.shape) > 1:
        d = np.sort(np.std(dat, axis=0))[::-1]
    else:
        d = np.sort(dat)[::-1]

    if threshold is not False:
        d = d[d > threshold]

    p = len(d)
    if p == 0:
        raise ValueError(f"d must have elements that are larger than the threshold {threshold}!")

    lq = np.zeros(p)  # log likelihood, function of q
    for q in range(p):
        mu1 = np.mean(d[:q+1])
        mu2 = np.mean(d[q+1:]) if q < p-1 else np.nan
        sigma2 = (np.sum((d[:q+1] - mu1)**2) + np.sum((d[q+1:] - mu2)**2)) / (p - 1 - (q < p-1))
        lq[q] = (np.sum(norm.logpdf(d[:q+1], mu1, np.sqrt(sigma2))) + 
                 np.sum(norm.logpdf(d[q+1:], mu2, np.sqrt(sigma2))))

    q = [np.argmax(lq)]
    if n > 1 and q[0] < (p - 1):
        q.extend([q[0] + 1 + el for el in get_elbows(d[q[0]+1:], n-1, plot=False)])

    if plot:
        if isinstance(dat, np.ndarray) and len(dat.shape) > 1:
            sdv = d
            plt.plot(sdv, marker='o')
            plt.xlabel("dim")
            plt.ylabel("stdev")
            plt.title(main)
            plt.scatter(q, sdv[q], s=100, color='red')
        else:
            plt.plot(dat, marker='o')
            plt.title(main)
            plt.scatter(q, dat[q], s=100, color='red')
        plt.show()

    return q



In [2]:
import os
import pickle as pkl
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
from graspologic.embed import AdjacencySpectralEmbed, LaplacianSpectralEmbed
from mpl_toolkits.mplot3d import Axes3D

# NOTE: The functions get_elbows() and filter_matrix_TC() must be defined 
# and available in your environment for the code to run fully.

# ===================================================================
# 1. PATH AND CONTROL SETUP
# ===================================================================

# --- ABSOLUTE PATHS ---
SAVE_ROOT_DR = r'/cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/'
DATA_ROOT_DR = r'/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/'

os.chdir(SAVE_ROOT_DR) 

# --- METHOD SELECTION CONTROL ---
EMBEDDING_METHOD = "ASE" # <--- CHANGE THIS TO "LSE" WHEN NEEDED

# --- WELLS TO PROCESS ---
WELLS_TO_PROCESS = [f"well{i:03d}" for i in range(6)] # well000 to well005

# --- CONSTANTS ---
TIME_SERIES_PREFIX_DIV = 'DIV' 

# --- FOLDER NAMING ---
ANALYSIS_TYPE = "TimeCourse_Embed"
OUTPUT_DIR_NAME = f"Python_{ANALYSIS_TYPE}_outputs_{EMBEDDING_METHOD}"

OUTPUT_DIR = os.path.join(SAVE_ROOT_DR, OUTPUT_DIR_NAME)
os.makedirs(OUTPUT_DIR, exist_ok=True) 


# --- EMBEDDING FUNCTION MAPPING ---
if EMBEDDING_METHOD == "LSE":
    EmbeddingClass = LaplacianSpectralEmbed
    METHOD_TITLE = "LSE"
else:
    EmbeddingClass = AdjacencySpectralEmbed
    METHOD_TITLE = "ASE"
embedder = EmbeddingClass(n_components=10, check_lcc=False)


# ===================================================================
# 2. FILE PATH MANIPULATION (Runs only once)
# ===================================================================

# Construct the absolute path for the inner directory structure
#dr1_relative = 'Raw Data Week 5.5 to 8.5 (run 8 and LD)/Run 8/'
dr1_relative = 'Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/'

dr1 = os.path.join(DATA_ROOT_DR, dr1_relative) 

# --- Extract Prefix for Figure Title ---
prefix_parts = dr1_relative.strip('/').split(TIME_SERIES_PREFIX_DIV)
TITLE_PREFIX_RAW = prefix_parts[0].strip('/_ ') 

filenames=os.listdir(dr1)
filenames.pop(2)
sorted_filenames = sorted(filenames, key=lambda x: int(x.split('DIV ')[1]))

# The list of file paths used for loading must be absolute
sorted_filenames_updated = [
    os.path.join(dr1, filename, 'data.raw_20240521_17h07m.pkl') 
    for filename in sorted_filenames
]

# Generate titles: DIV 3, DIV 5, ..., DIV 21
div_titles = [f"DIV {x}" for x in range(3, 23, 2)]

print(f"Data file paths prepared. {len(sorted_filenames_updated)} time points found.")
print(f"Ready to process {len(WELLS_TO_PROCESS)} wells.")

Data file paths prepared. 10 time points found.
Ready to process 6 wells.


In [4]:
sorted_filenames_updated

['/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/DIV 3/data.raw_20240521_17h07m.pkl',
 '/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/DIV 5/data.raw_20240521_17h07m.pkl',
 '/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/DIV 7/data.raw_20240521_17h07m.pkl',
 '/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/DIV 9/data.raw_20240521_17h07m.pkl',
 '/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/DIV 11/data.raw_20240521_17h07m.pkl',
 '/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_resu

In [9]:
# ===================================================================
# 3. MAIN PLOTTING AND SAVING LOOP (Iterates over all wells)
# ===================================================================

for WELL_NAME in WELLS_TO_PROCESS:
    
    # --- Data Loading (Runs for the current WELL_NAME) ---
    all_adj = []
    for file_path in sorted_filenames_updated:
        with open(file_path, 'rb') as f:
            data = pkl.load(f)
        # filter_matrix_TC must be defined
        adj = filter_matrix_TC(data, WELL_NAME) 
        all_adj.append(adj)

    # --- Filename and Title Construction ---
    # The figure title structure: [Prefix] | [Well] | [Method]
    fig_suptitle = f"{TITLE_PREFIX_RAW} | {WELL_NAME} | {METHOD_TITLE}"

    # The filename is derived from the figure title (cleaned)
    full_filename_base = (
        fig_suptitle.replace("/", "_")
        .replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("-", "_")
        .replace("|", "")
        + ".pdf"
    )
    final_pdf_path = os.path.join(OUTPUT_DIR, full_filename_base)

    # Create one large figure for the 2x5 grid
    fig = plt.figure(figsize=(20, 8)) 
    fig.suptitle(fig_suptitle, fontsize=20, fontweight='bold', y=0.98)


    # Limit loop to 10 items to fit the 2x5 grid (DIV 3 to 21)
    for i in range(min(len(all_adj), 10)):
        
        adj = all_adj[i]
        current_title = f"{div_titles[i]}"
        
        # 1. Symmetrize & Extract LCC
        adj_symm = ((adj + adj.T) > 0).astype(int)
        g = ig.Graph.Adjacency(adj_symm.tolist(), mode="undirected")
        lcc = g.connected_components().giant() 
        
        # 2. Check LCC size
        if lcc.vcount() < 11:
            ax = fig.add_subplot(2, 5, i + 1)
            ax.text(0.5, 0.5, "LCC < 11", ha='center', va='center', color='red')
            ax.set_title(current_title, fontsize=12)
            ax.axis('off')
            continue

        # 3. Run Embedding (ASE or LSE)
        mat_lcc = np.array(lcc.get_adjacency().data)
        
        try:
            Xhat = embedder.fit_transform(mat_lcc)
        except ValueError:
            ax = fig.add_subplot(2, 5, i + 1)
            ax.text(0.5, 0.5, f"{METHOD_TITLE} Failed", ha='center', va='center', color='red')
            ax.axis('off')
            continue

        # 4. Determine Dimension
        embedding_dim = 3 # Forced to 3

        # 5. Dynamic Plotting (2D vs 3D)
        if embedding_dim >= 3:
            ax = fig.add_subplot(2, 5, i + 1, projection='3d')
            ax.scatter(Xhat[:, 0], Xhat[:, 1], Xhat[:, 2], c='blue', alpha=0.6, s=15)
            ax.set_xlabel("Dim 1", fontsize=8)
            ax.set_ylabel("Dim 2", fontsize=8)
            ax.set_zlabel("Dim 3", fontsize=8)
        else:
            ax = fig.add_subplot(2, 5, i + 1)
            
            if embedding_dim < 2:
                ax.text(0.5, 0.5, f"Dim < 2 ({embedding_dim})", color="orange", ha='center')
                ax.axis('off')
            else:
                ax.scatter(Xhat[:, 0], Xhat[:, 1], c='blue', alpha=0.6, s=15)
                ax.set_xlabel("Dimension 1", fontsize=8)
                ax.set_ylabel("Dimension 2", fontsize=8)

        ax.set_title(current_title, fontsize=12, fontweight='bold')

    plt.tight_layout(rect=[0, 0.03, 1, 0.93])

    # 6. --- SAVE FIGURE ---
    print(f"Saving figure to: {final_pdf_path}")
    plt.savefig(final_pdf_path, format='pdf', bbox_inches='tight')
    plt.close(fig)

print("\nAll well figures processed and saved.")

Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_Embed_outputs_ASE/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well000__ASE.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_Embed_outputs_ASE/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well001__ASE.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_Embed_outputs_ASE/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well002__ASE.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_Embed_outputs_ASE/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well003__ASE.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_Embed_outputs_ASE/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well004__ASE.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_Embed_outputs_ASE/Raw_Data_Week_9.5_to_12.5_run_6_and_

In [3]:
## This shows that the data doesn't have window 
with open(sorted_filenames_updated[0], 'rb') as f:
    data = pkl.load(f)
data['well000'].keys()

dict_keys(['spike_amp_thresh', 'channel_numbers', 'channel_spikes_per_sec', 'win_0'])

In [4]:
data['config']

{'paths': {'source_files': '/aoscluster/moped/data/brain_organoid_spikes/TimeCourseData',
  'results': '/aoscluster/moped/data/brain_organoid_spikes/TimeCourseData_ecr_results'},
 'data': {'fs': 10000,
  'spike_amp_thresh_percentile': 5,
  'corr_amp_thresh_percentile': None,
  'corr_amp_thresh_std': 1},
 'windows': {'win_dur': 'None'},
 'super_sel': {'recompute': False,
  'adj_threshold': 1.0,
  'raster_dur': 0.0005,
  'corr_type': 'cc',
  'n_corr_peaks_max': 4,
  'epsilon': 0.003,
  'T_list': [0.02, 0.0175, 0.016],
  'sigma_list': [0.0004, 0.00055, 0.0007]}}

## Directed CP Stats Analysis 


In [2]:
import os
import pickle as pkl
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

# ===================================================================
# 1. PATH AND CONTROL SETUP
# ===================================================================

# --- ABSOLUTE PATHS ---
# (Adjust these paths as needed for your specific environment)
SAVE_ROOT_DR = r'/cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/'
DATA_ROOT_DR = r'/cis/project/organoid/2024May28 No window data /OneDrive_1_6-17-2024/TimeCourseData_ecr_results/'

try:
    os.chdir(SAVE_ROOT_DR)
except FileNotFoundError:
    print(f"Warning: Root directory {SAVE_ROOT_DR} not found. Running in current directory.")

# --- WELLS TO PROCESS ---
WELLS_TO_PROCESS = [f"well{i:03d}" for i in range(6)] 

# --- CONSTANTS ---
TIME_SERIES_PREFIX_DIV = 'DIV' 

# --- FOLDER NAMING ---
ANALYSIS_TYPE = "TimeCourse_DIRECTED_CP_Stats"
OUTPUT_DIR_NAME = f"Python_{ANALYSIS_TYPE}_outputs"

OUTPUT_DIR = os.path.join(SAVE_ROOT_DR, OUTPUT_DIR_NAME)
os.makedirs(OUTPUT_DIR, exist_ok=True) 

print("Environment setup complete.")

Environment setup complete.


In [13]:
# ===================================================================
# 2. HELPER FUNCTION: Directed CP Statistics + Reciprocity
# ===================================================================

def get_directed_clique_density_stats(adj):
    """
    Calculates Core-Periphery statistics for a DIRECTED graph, including
    densities and reciprocity (rho) for the whole graph and per block.
    """
    # Ensure binary directed adjacency
    adj_bin = (adj > 0).astype(int)
    
    # Create Directed Graph
    g = ig.Graph.Adjacency(adj_bin.tolist(), mode="directed")
    
    # Coreness (mode='all' for structural coreness)
    kc = np.array(g.coreness(mode="all"))
    max_k = np.max(kc) if len(kc) > 0 else 0
    
    core_indices = np.where(kc == max_k)[0]
    all_indices = np.arange(adj.shape[0])
    periph_indices = np.setdiff1d(all_indices, core_indices)
    
    n_c, n_p = len(core_indices), len(periph_indices)
    
    # --- Extract Sub-matrices ---
    # Rows are Source, Cols are Target
    Acc = adj[np.ix_(core_indices, core_indices)] if n_c > 0 else np.array([])
    App = adj[np.ix_(periph_indices, periph_indices)] if n_p > 0 else np.array([])
    
    # Acp: Core -> Periph
    Acp = adj[np.ix_(core_indices, periph_indices)] if (n_c > 0 and n_p > 0) else np.array([])
    # Apc: Periph -> Core
    Apc = adj[np.ix_(periph_indices, core_indices)] if (n_c > 0 and n_p > 0) else np.array([])

    # --- 1. EDGE COUNTS (Sums) ---
    edges_CC = Acc.sum() if Acc.size > 0 else 0
    edges_PP = App.sum() if App.size > 0 else 0
    edges_CP = Acp.sum() if Acp.size > 0 else 0
    edges_PC = Apc.sum() if Apc.size > 0 else 0
    
    total_edges = edges_CC + edges_PP + edges_CP + edges_PC

    # --- 2. DENSITIES ---
    max_CC = n_c * (n_c - 1)
    max_PP = n_p * (n_p - 1)
    max_CP = n_c * n_p
    max_PC = n_p * n_c # Same as CP
    
    dens_CC = edges_CC / max_CC if max_CC > 0 else 0
    dens_PP = edges_PP / max_PP if max_PP > 0 else 0
    dens_CP = edges_CP / max_CP if max_CP > 0 else 0
    dens_PC = edges_PC / max_PC if max_PC > 0 else 0

    # --- 3. RECIPROCITY (RHO) CALCULATIONS ---
    # Formula: sum(A * A.T) / sum(A)
    # This counts *endpoints* of mutual edges. If i<->j exists, it contributes 2 to num, 2 to denom.
    
    # A. Global Reciprocity
    num_rho_global = np.sum(adj * adj.T)
    den_rho_global = np.sum(adj)
    rho_global = num_rho_global / den_rho_global if den_rho_global > 0 else 0

    # B. Block-wise Reciprocities
    
    # Rho CC: (Mutuals inside Core) / (Edges inside Core)
    rho_CC = 0
    if Acc.size > 0 and edges_CC > 0:
        rho_CC = np.sum(Acc * Acc.T) / edges_CC
        
    # Rho PP: (Mutuals inside Periph) / (Edges inside Periph)
    rho_PP = 0
    if App.size > 0 and edges_PP > 0:
        rho_PP = np.sum(App * App.T) / edges_PP

    # Rho Inter-Block (CP and PC interface)
    # Mutuals here are edges where (c->p) AND (p->c) exist.
    # Numerator: 2 * count of mutual pairs.
    # We can calculate this by interacting Acp and Apc.
    # Note: Acp[i,j] corresponds to Core_i -> Periph_j.
    #       Apc[j,i] corresponds to Periph_j -> Core_i.
    #       So we check element-wise: Acp * (Apc Transposed)
    rho_inter = 0
    edges_inter = edges_CP + edges_PC
    if Acp.size > 0 and edges_inter > 0:
        # Acp is (Nc x Np). Apc is (Np x Nc). Apc.T is (Nc x Np).
        # Element-wise mult finds where both exist.
        mutual_inter_matrix = Acp * Apc.T 
        # Sum gives count of mutual PAIRS (from perspective of C->P).
        # Each pair implies 2 mutual endpoints (one in C, one in P).
        # Formula consistency: sum(endpoints) / sum(all edges)
        rho_inter = (2 * np.sum(mutual_inter_matrix)) / edges_inter

    return {
        'dens_CC': dens_CC, 'dens_CP': dens_CP,
        'dens_PC': dens_PC, 'dens_PP': dens_PP,
        'n_core': n_c, 'n_periph': n_p,
        # Reciprocities
        'rho_global': rho_global,
        'rho_CC': rho_CC,
        'rho_PP': rho_PP,
        'rho_CP': rho_inter, # CP and PC share the same reciprocity score
        'rho_PC': rho_inter
    }

print("Helper function updated with Block Reciprocities.")

Helper function updated with Block Reciprocities.


In [16]:
# ===================================================================
# 3. FILE PATH PREPARATION
# ===================================================================

# Note: Adjust relative path as per your specific folder structure
#dr1_relative = 'Raw Data Week 5.5 to 8.5 (run 8 and LD)/Run 8/'
dr1_relative = 'Raw Data Week 9.5 to 12.5 (run 6 and 7)/Run 7/'
dr1 = os.path.join(DATA_ROOT_DR, dr1_relative) 

# Robust safe-check for file listing
if os.path.exists(dr1):
    filenames = os.listdir(dr1)
    # Replicating your logic of popping the 3rd element if needed (index 2)
    if len(filenames) > 2: filenames.pop(2)
    
    # Sorting logic
    try:
        sorted_filenames = sorted(filenames, key=lambda x: int(x.split('DIV ')[1]))
        
        sorted_filenames_updated = [
            os.path.join(dr1, f, 'data.raw_20240521_17h07m.pkl') for f in sorted_filenames
        ]
        
        div_titles = [f"DIV {x}" for x in range(3, 23, 2)]
        
        # Extract title prefix
        prefix_parts = dr1_relative.strip('/').split(TIME_SERIES_PREFIX_DIV)
        TITLE_PREFIX_RAW = prefix_parts[0].strip('/_ ') 
        
        print(f"Setup complete. Found {len(sorted_filenames_updated)} files.")
        print(f"Ready to process {len(WELLS_TO_PROCESS)} wells: {WELLS_TO_PROCESS}")
        
    except Exception as e:
        print(f"Error during file sorting: {e}")
        sorted_filenames_updated = []
        div_titles = []
        TITLE_PREFIX_RAW = "Unknown_Dataset"
else:
    print(f"Directory not found: {dr1}")
    sorted_filenames_updated = []
    div_titles = []
    TITLE_PREFIX_RAW = "Unknown"

Setup complete. Found 10 files.
Ready to process 6 wells: ['well000', 'well001', 'well002', 'well003', 'well004', 'well005']


In [17]:
# ===================================================================
# 4. MAIN CP STATS PLOTTING AND SAVING LOOP (Revised)
# ===================================================================

for WELL_NAME in WELLS_TO_PROCESS:
    
    # --- Load Data for the current well ---
    all_adj = []
    if not sorted_filenames_updated:
        print("No files to process.")
        break
        
    for file_path in sorted_filenames_updated:
        with open(file_path, 'rb') as f:
            data = pkl.load(f)
        # Using existing filter function
        adj = filter_matrix_TC(data, WELL_NAME) 
        all_adj.append(adj)

    # --- Title and Filename Logic ---
    fig_suptitle = f"{TITLE_PREFIX_RAW} | {WELL_NAME} | DIRECTED CP_STATS"
    full_filename = (fig_suptitle.replace("/", "_").replace(" ", "_")
                     .replace("(", "").replace(")", "").replace("-", "_")
                     .replace("|", "") + ".pdf")
    final_pdf_path = os.path.join(OUTPUT_DIR, full_filename)

    # Setup Figure: 2x5 grid
    fig = plt.figure(figsize=(22, 12))
    fig.suptitle(fig_suptitle, fontsize=20, fontweight='bold', y=0.98)

    for i in range(min(len(all_adj), 10)):
        adj = all_adj[i]
        current_title = div_titles[i] if i < len(div_titles) else f"Time {i}"
        ax = fig.add_subplot(2, 5, i + 1)
        
        # --- 1. Directed Graph Creation ---
        adj_bin = (adj > 0).astype(int)
        g = ig.Graph.Adjacency(adj_bin.tolist(), mode="directed")
        
        # --- 2. LCC (Weakly Connected) ---
        lcc_g = g.components(mode="weak").giant()
        n_original, n_lcc = g.vcount(), lcc_g.vcount()
        
        # --- 3. Size Threshold ---
        if n_lcc < 11:
            ax.text(0.5, 0.5, "LCC size < 11", color="red", ha='center', va='center', fontsize=12)
            ax.axis('off')
            ax.set_title(current_title, fontweight='bold')
            continue

        # --- 4. Calculate Stats ---
        mat_lcc = np.array(lcc_g.get_adjacency().data)
        
        try:
            # The function now calculates ALL rhos internally
            stats = get_directed_clique_density_stats(mat_lcc)
            
            # Format Text
            text_str = (
                "--- Densities (Dir) ---\n"
                f"C -> C: {stats['dens_CC']:.3f}\n"
                f"C -> P: {stats['dens_CP']:.3f}\n"
                f"P -> C: {stats['dens_PC']:.3f}\n"
                f"P -> P: {stats['dens_PP']:.3f}\n\n"
                
                "--- Block Symmetry ---\n"
                f"Rho CC: {stats['rho_CC']:.3f}\n"
                f"Rho CP: {stats['rho_CP']:.3f}\n"
                f"Rho PC: {stats['rho_PC']:.3f}\n"
                f"Rho PP: {stats['rho_PP']:.3f}\n\n"

                "--- Counts ---\n"
                f"Core: {stats['n_core']}\n"
                f"Peri: {stats['n_periph']}\n"
                f"LCC Size: {n_lcc}\n"
                f"Orig Size: {n_original}\n\n"
                                
                "--- Global Symmetry ---\n"
                f"Mutual/All: {stats['rho_global']:.3f}\n\n"
                
                "--- Ratios ---\n"
                f"Core/LCC: {stats['n_core']/n_lcc:.3f}\n"
                f"Peri/LCC: {stats['n_periph']/n_lcc:.3f}"
            )
            ax.text(0.5, 0.5, text_str, ha='center', va='center', family='monospace', fontsize=11)
            
        except Exception as e:
            ax.text(0.5, 0.5, f"Error: {str(e)}", color='red', ha='center')

        ax.axis('off')
        ax.set_title(current_title, fontweight='bold', fontsize=14)

    plt.tight_layout(rect=[0, 0.03, 1, 0.93])
    
    print(f"Saving figure to: {final_pdf_path}")
    plt.savefig(final_pdf_path, format='pdf', bbox_inches='tight')
    plt.close(fig)

print("\nAll DIRECTED CP Stats well figures processed successfully.")

Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_DIRECTED_CP_Stats_outputs/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well000__DIRECTED_CP_STATS.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_DIRECTED_CP_Stats_outputs/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well001__DIRECTED_CP_STATS.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_DIRECTED_CP_Stats_outputs/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well002__DIRECTED_CP_STATS.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_DIRECTED_CP_Stats_outputs/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well003__DIRECTED_CP_STATS.pdf
Saving figure to: /cis/home/tchen94/tianyi/Organoid/Time course LTP(END)/Python_TimeCourse_DIRECTED_CP_Stats_outputs/Raw_Data_Week_9.5_to_12.5_run_6_and_7_Run_7__well004__DIRECTED_CP_STATS.pdf
Saving figure to: /cis/home/tchen94